In [9]:
pip install --upgrade keras


In [10]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

In [11]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install --upgrade keras


from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
import tensorflow.keras as keras




In [12]:
data = pd.read_csv('DataTrain.csv')
# Keeping only the neccessary columns
data = data[['text','sentiment']]


In [13]:
# Filter and keep rows where no cell in any column contains a string with length > 12
data = data[~data.applymap(lambda x: isinstance(x, str) and len(x) > 12).any(axis=1)]

# Reset the index after dropping rows
data.reset_index(drop=True, inplace=True)

# Print the resulting DataFrame
print(data)

             text sentiment
0            Kpai  positive
1     Really Good  positive
2       Not Bad..  positive
3      Perfect 👌👌  positive
4         Usefull  positive
..            ...       ...
794          heyy   neutral
795      Its okay  negative
796    Great idea  positive
797  Back to work   neutral
798      Surgery.   neutral

[799 rows x 2 columns]


In [14]:
X = data['text']

# Tokenize your text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Find the maximum sequence length
max_sequence_length = max(len(seq) for seq in X)

# Pad sequences to a fixed length (use max_sequence_length as input_length)
X = pad_sequences(X, maxlen=max_sequence_length)

In [15]:
embed_dim = 128
lstm_out = 196
max_features = 10000  # You can adjust this value based on your dataset size and vocabulary.

model = Sequential()
model.add(Embedding(max_features, embed_dim, input_length=X.shape[1]))  # Corrected max_features
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())





Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 5, 128)            1280000   
                                                                 
 spatial_dropout1d (Spatial  (None, 5, 128)            0         
 Dropout1D)                                                      
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 2)                 394       
                                                                 
Total params: 1535194 (5.86 MB)
Trainable params: 1535194 (5.86 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [16]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [17]:
stopwords = stopwords.words('english')
stemmer = SnowballStemmer('english')

In [18]:
import re
# using regex to remove hyperlinks and other using links
text_clean = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

In [19]:
df1 = data.copy()
x = df1.text[3]
x

'Perfect 👌👌'

In [20]:
xx = re.sub(text_clean, ' ', str(x).lower()).strip()
z = []
for token in xx.split():
   z.append(token)
print (z)
" ".join(z)

['perfect']


'perfect'

In [21]:
for i, words in enumerate(stopwords):
  if words=="good":
    print (i, words)
    break

In [22]:
# function to cleanup the text
def preprocess(text, stem=False):
  text = re.sub(text_clean, ' ', str(text).lower()).strip()
  tokens = []
  for token in text.split():
    if token not in stopwords:
      if stem:
        tokens.append(stemmer.stem(token))
      else:
        tokens.append(token)
  return " ".join(tokens)

In [23]:
# Filter and keep rows where no cell in any column contains a string with length > 12
df1 = df1[~df1.applymap(lambda x: isinstance(x, str) and len(x) > 12).any(axis=1)]

# Reset the index after dropping rows
df1.reset_index(drop=True, inplace=True)

# Print the resulting DataFrame
print(df1)

             text sentiment
0            Kpai  positive
1     Really Good  positive
2       Not Bad..  positive
3      Perfect 👌👌  positive
4         Usefull  positive
..            ...       ...
794          heyy   neutral
795      Its okay  negative
796    Great idea  positive
797  Back to work   neutral
798      Surgery.   neutral

[799 rows x 2 columns]


In [24]:
df1 = data.copy()
df1.text = df1.text.apply(lambda x: preprocess(x))
df1["text"]
df=df1

In [25]:
# Importing required libraries
import nltk
import pandas as pd
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [26]:
#Check if there are any null values
# Select the desired columns
df = df[['text', 'sentiment']]

df.isnull().sum()

text         0
sentiment    0
dtype: int64

In [27]:
# Encoded the target column
lb=LabelEncoder()
df['sentiment'] = lb.fit_transform(df['sentiment'])


In [28]:
tokenizer = Tokenizer(num_words=500, split=' ')
tokenizer.fit_on_texts(df['text'].values)
X = tokenizer.texts_to_sequences(df['text'].values)
X = pad_sequences(X)

In [29]:
model = Sequential()
model.add(Embedding(500, 120, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 5, 120)            60000     
                                                                 
 spatial_dropout1d_1 (Spati  (None, 5, 120)            0         
 alDropout1D)                                                    
                                                                 
 lstm_1 (LSTM)               (None, 176)               209088    
                                                                 
 dense_1 (Dense)             (None, 3)                 531       
                                                                 
Total params: 269619 (1.03 MB)
Trainable params: 269619 (1.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [30]:
#Splitting the data into training and testing
y=pd.get_dummies(df['sentiment'])
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.3, random_state = 42)

In [31]:
from keras.utils import to_categorical

# Convert integer labels to one-hot encoded labels
y_train_encoded = to_categorical(y_train, num_classes=3)


In [32]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [33]:
model.fit(X_train,y_train,epochs=10,batch_size=32,verbose='auto')


Epoch 1/10
18/18 [==============================] - 5s 54ms/step - loss: 1.0695 - accuracy: 0.4347
Epoch 2/10
18/18 [==============================] - 1s 46ms/step - loss: 1.0383 - accuracy: 0.4597
Epoch 3/10
18/18 [==============================] - 1s 53ms/step - loss: 1.0090 - accuracy: 0.4687
Epoch 4/10
18/18 [==============================] - 1s 52ms/step - loss: 0.9531 - accuracy: 0.5581
Epoch 5/10
18/18 [==============================] - 1s 50ms/step - loss: 0.8186 - accuracy: 0.6673
Epoch 6/10
18/18 [==============================] - 1s 60ms/step - loss: 0.6503 - accuracy: 0.7460
Epoch 7/10
18/18 [==============================] - 1s 54ms/step - loss: 0.5222 - accuracy: 0.8497
Epoch 8/10
18/18 [==============================] - 1s 83ms/step - loss: 0.3953 - accuracy: 0.9088
Epoch 9/10
18/18 [==============================] - 2s 84ms/step - loss: 0.3267 - accuracy: 0.9141
Epoch 10/10
18/18 [==============================] - 1s 70ms/step - loss: 0.2897 - accuracy: 0.9141
